In [1]:
#import library and module
import pandas as pd
from resume_screening import resparser, match

c:\Users\Acer\anaconda3\envs\resume-parser\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
from nltk.corpus import stopwords
stopw  = set(stopwords.words('english'))

In [3]:
#read indeed job vacancy data
job = pd.read_csv('indeed_data.csv')
job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))

In [4]:
df = job.drop_duplicates(subset='test').reset_index(drop=True)
df.head()

,title,company,salary,description,link,test
0,Data Scientist (Analytics),Grab,None,"Job Description: Life at Grab At Grab, every G...",https://id.indeed.com/rc/clk?jk=acd6479660d3fc...,"Job Description: Life Grab Grab, every Grabber..."
1,Data Scientist,Sayurbox,None,Job Description Be a part of Sayurbox data tea...,https://id.indeed.com/rc/clk?jk=40482050a5e4d8...,Job Description part Sayurbox data team target...
2,Senior Data Scientist - GoPay,GO-JEK,None,Location Jakarta Work Type Permanent Applicati...,https://id.indeed.com/rc/clk?jk=60463a42bb19a6...,Location Jakarta Work Type Permanent Applicati...
3,Data Scientist,Allianz Indonesia,None,Bachelor Degree with field of study mathematic...,https://id.indeed.com/rc/clk?jk=835f2c980867fe...,"Bachelor Degree field study mathematic, inform..."
4,Senior Data Scientist,Sayurbox,None,Job Description: Build Sayurbox data team acco...,https://id.indeed.com/rc/clk?jk=da6afcc2ee471d...,Job Description: Build Sayurbox data team acco...


In [5]:
test = (df['test'].values.astype('U'))

In [6]:
print("There are {} job vacancy list scrapped from id.indeed.com".format(df.shape[0]))

There are 567 job vacancy list scrapped from id.indeed.com


In [7]:
#input the resume or CV file
filled = input()
skills = resparser.skill("resume_files/{}".format(filled))

In [8]:
skills.append(match.preprocessing(skills[0]))
del skills[0]

print("SKILLS LIST:\n",skills)

SKILLS LIST:
 ['online machine learning mobile applications prototype figma user experience design ued building banners user experience interview ug dsc content design interfaces wireframing investors design organization social media mastery web design internships gsm java ux design mobile google learning export higher education teaching experience design promotional feeds instagram options testing sketching media design training graphic design visual design space app linkedin moc digital usability testing developers materials user interface design ux illustration usability web manuals']


# Model KNN - Minkowski Metric

In [11]:
#Train the KNN model using Minkowski Metric
matches = match.knearestNeighbors(skills, test)
matches = pd.DataFrame(matches, columns=['Match confidence'])

In [17]:
#Job Vacancy Recommendation
result_minkowski = df[['title','company','salary']].copy()
result_minkowski['match'] = matches['Match confidence'].copy()
result_minkowski = result_minkowski.sort_values('match').reset_index(drop=True).head(10)
result_minkowski

,title,company,salary,match
0,Oracle DBA Consultant,NTT Ltd,None,0.72
1,DATABASE ADMINISTRATOR – Entertainment Platfor...,Parrish & Co.,None,0.73
2,Database Administrator,Mekari (PT. Mid Solusi Nusantara),None,0.74
3,Database Administrator Officer - Finance Services,RGF HR Agent,None,0.75
4,Database Administrator (for AMAAN Project),DKatalis,None,0.75
5,Database Administrator Officer,Reeracoen Indonesia,None,0.76
6,Oracle Database Administrator,PT. Berca Hardayaperkasa,None,0.77
7,Business Intelligence (Junior/Senior/Lead),Cermati.com,None,0.77
8,"Data Engineer, Commercial & Operation Team",ZALORA SOUTH EAST ASIA PTE LTD,None,0.77
9,System Administrator,Walden Global Services - WGS,None,0.77


# Model Cosine Similarity

In [13]:
df['clean'] = df['test'].apply(match.preprocessing)
jobdesc = (df['clean'].values.astype('U'))

In [14]:
count_matrix = match.vectorizing(skills[0], jobdesc)
matchPercentage = match.coSim(count_matrix)
matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

In [18]:
#Job Vacancy Recommendations
result_cosine = df[['title','company','salary']]
result_cosine = result_cosine.join(matchPercentage)
result_cosine = result_cosine.sort_values('Skills Match', ascending=False).reset_index(drop=True).head(10)
result_cosine

,title,company,salary,Skills Match
0,IT Database Administrator,PT Anabatic Digital Raya,None,32.57
1,CISO12 - Data Protection Officer,PT Bank Mandiri (Persero) Tbk.,None,31.25
2,Senior Database Administrator,Syscon Justice System,None,31.25
3,DBA Junior Level,Krakatau Information Technology,None,30.71
4,Database Administrator,PT. PRIMA VISTA SOLUSI,None,30.62
5,Oracle Database Administrator,NTT INDONESIA TECHNOLOGY,None,30.17
6,Business Intelligence (Tableau),NTT INDONESIA TECHNOLOGY,None,28.49
7,Oracle Database Administrator,PT. Berca Hardayaperkasa,None,28.18
8,CISO16 - Data Encryption & Key Management Officer,PT Bank Mandiri (Persero) Tbk.,None,28.17
9,DATABASE ADMINISTRATOR – Entertainment Platfor...,Parrish & Co.,None,27.54


# Comparing between Minkowski Metric vs Cosine Similarity

In [19]:
data = [result_minkowski[['title','company']], result_cosine[['title','company']]]
headers = ['Minkowski','Cosine Similarity']
df_compare = pd.concat(data, axis = 1, keys = headers)
df_compare

Minkowski  \
                                               title   
0                              Oracle DBA Consultant   
1  DATABASE ADMINISTRATOR – Entertainment Platfor...   
2                             Database Administrator   
3  Database Administrator Officer - Finance Services   
4         Database Administrator (for AMAAN Project)   
5                     Database Administrator Officer   
6                      Oracle Database Administrator   
7         Business Intelligence (Junior/Senior/Lead)   
8         Data Engineer, Commercial & Operation Team   
9                               System Administrator   

                                      \
                             company   
0                            NTT Ltd   
1                      Parrish & Co.   
2  Mekari (PT. Mid Solusi Nusantara)   
3                       RGF HR Agent   
4                           DKatalis   
5                Reeracoen Indonesia   
6           PT. Berca Hardayaperkasa   
7                        Cermati.com   
8     ZALORA SOUTH EAST ASIA PTE LTD   
9       Walden Global Services - WGS   

                                   Cosine Similarity  \
                                               title   
0                          IT Database Administrator   
1                   CISO12 - Data Protection Officer   
2                      Senior Database Administrator   
3                                   DBA Junior Level   
4                             Database Administrator   
5                      Oracle Database Administrator   
6                    Business Intelligence (Tableau)   
7                      Oracle Database Administrator   
8  CISO16 - Data Encryption & Key Management Officer   
9  DATABASE ADMINISTRATOR – Entertainment Platfor...   

                                    
                           company  
0         PT Anabatic Digital Raya  
1   PT Bank Mandiri (Persero) Tbk.  
2            Syscon Justice System  
3  Krakatau Information Technology  
4           PT. PRIMA VISTA SOLUSI  
5         NTT INDONESIA TECHNOLOGY  
6         NTT INDONESIA TECHNOLOGY  
7         PT. Berca Hardayaperkasa  
8   PT Bank Mandiri (Persero) Tbk.  
9                    Parrish & Co.

In [20]:
job_compare = pd.merge(df_compare['Minkowski'], df_compare['Cosine Similarity'], how = 'inner')
job_compare

,title,company
0,DATABASE ADMINISTRATOR – Entertainment Platfor...,Parrish & Co.
1,Oracle Database Administrator,PT. Berca Hardayaperkasa


In [21]:
similarityPercentage = (job_compare.shape[0]/df_compare.shape[0])*100
print("Similarity between Minkowski Model and Cosine Similarity Model is {} %".format(similarityPercentage))

Similarity between Minkowski Model and Cosine Similarity Model is 20.0 %
